# Checkpoint 3 — Intenção "ingredientes" e integração com Deepseek

Objetivo
- Aumentar a aplicação NLP do desafio 1 adicionando a intenção "ingredientes".
- Quando a intenção "ingredientes" for detectada, pedir que o usuário escolha o nome do prato (um dentre os disponíveis), validar a escolha e então consultar a Deepseek para retornar a receita/ingredientes ao usuário.

Prazos e observações
- Vale 10 pontos.
- Entrega até 07/11.
- Sessão de dúvidas: 31/10.
- Alunos que não entregaram o Checkpoint 2: criar uma lista de pratos selecionáveis, permitir seleção de um ou mais pratos e trazer a receita de cada prato selecionado.

Regras funcionais (mínimo obrigatório)
1. Detectar a intenção "ingredientes" (p. ex. palavras/frases como: "quais são os ingredientes", "preciso dos ingredientes de", "o que tem no prato", "buscar ingredientes", etc.).
2. Ao detectar essa intenção:
   - Pedir ao usuário qual prato ele quer (exibir lista de pratos disponíveis).
   - Validar que a seleção pertence à lista. Se inválida, pedir novamente ou exibir opções.
   - Enviar uma requisição à Deepseek perguntando "qual a receita do prato <nome_prato>?" (ou "quais são os ingredientes do <nome_prato>?").
   - Mostrar a receita/ingredientes retornados ao usuário.
3. Suportar seleção múltipla para quem não entregou Checkpoint 2 (ex.: entrada de índices separados por vírgula).

Entrega
- Arquivo notebook (.ipynb) atualizado com as células que implementam a intenção e a integração.
- Breve README (ou célula markdown no notebook) explicando como executar e onde colocar a API key.

Critério de avaliação (exemplo)
- Detecção da intenção e cobertura de frases: 3 pontos
- Integração correta com Deepseek e tratamento de respostas: 3 pontos
- Funcionalidade (eu conseguir executar o mesmo código com minha própria chave API): 2 pontos
- Todas intenções do checkpoint 2: 2 pontos

Exemplo de implementação(apenas para referência)
```python
import os
import requests
import json

PRATOS = ["lasanha", "feijoada", "moqueca", "spaghetti alla carbonara"]

INTENT_INGREDIENTES = ["ingredientes", "quais são os ingredientes", "o que tem", "buscar ingredientes", "receita do prato"]

def detect_intent(text: str) -> str:
    txt = text.lower()
    for kw in INTENT_INGREDIENTES:
        if kw in txt:
            return "ingredientes"
    return "outro"

def escolha_prato_interativa():
    print("Pratos disponíveis:")
    for i, p in enumerate(PRATOS, start=1):
        print(f"{i}. {p}")
    escolha = input("Escolha o número do prato (ou vários números separados por vírgula): ").strip()
    indices = [s.strip() for s in escolha.split(",") if s.strip().isdigit()]
    selecionados = []
    for idx in indices:
        i = int(idx) - 1
        if 0 <= i < len(PRATOS):
            selecionados.append(PRATOS[i])
    if not selecionados:
        print("Seleção inválida. Tente novamente.")
        return escolha_prato_interativa()
    return selecionados

def consulta_deepseek(nome_prato: str, api_key: str):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    prompt = f"Qual a receita do prato {nome_prato}?" #mudar essa frase OBRIGATORIAMENTE
    payload = {
        "model": "deepseek/deepseek-r1:free",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    resp = requests.post(url, headers=headers, json=payload, timeout=20)
    try:
        j = resp.json()
        # Ajuste conforme o formato retornado; exemplo genérico:
        content = j.get("choices", [{}])[0].get("message", {}).get("content")
        return content or json.dumps(j, ensure_ascii=False, indent=2)
    except Exception:
        return f"Erro na resposta da API: {resp.text}"

# Exemplo de fluxo
if __name__ == "__main__":
    user_input = input("Usuário: ")
    intent = detect_intent(user_input)
    if intent == "ingredientes":
        pratos_escolhidos = escolha_prato_interativa()  # lista de nomes
        api_key = os.getenv("DEEPSEEK_API_KEY") or input("Insira a API key da Deepseek: ").strip()
        for prato in pratos_escolhidos:
            print(f"\nBuscando receita para: {prato}\n")
            resultado = consulta_deepseek(prato, api_key)
            print(resultado)
    else:
        print("Intent não reconhecida no exemplo. Integre com seu NLU.")
```

- Não envie sua API Key no documento de resposta.
